In [ ]:
// #r ".\binaries2\bossspad.dll"
// #r ".\binaries2\XNSEC.dll"
// #r "C:\BoSSS_Binaries\bossspad.dll"
// #r "C:\BoSSS_Binaries\XNSEC.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;

using BoSSS.Application.XNSEC;

Init();

## Case configuration

In [ ]:
int[] dgDegree = new int[]{1,2,3,4};

int[] Resolutions = new int[]{3,4,5,6,7,8,9};

int numberOfMpiCores =4;
var  NofCells = Resolutions.ToList().Select(r=> (int)Math.Pow(2,r)).ToArray();
string baseJobName = "ChamberedFlame";

In [ ]:
NofCells

index,value
0,8
1,16
2,32
3,64
4,128
5,256
6,512


## Open Database

In [ ]:

// static var myDb = OpenOrCreateDatabase(@"C:\Databases\ChamberedDiffFlame5"); 
static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\UnstrainedFlame_100ItMax_unitylew_noVariableKin_ConstantCP_highervel"); 

// static string dirname ="CounterDiffusionFlame_HLLC222";
// static string winpath = @"S:\work\scratch\jg11bano\"+dirname;
// static string remotepath = @"/work/scratch/jg11bano/"+dirname;


// static var myDb = OpenOrCreateDatabase(@"S:\work\scratch\jg11bano\CDF_FirstTryHLLC");

// static var myDb = OpenOrCreateDatabase(winpath);
myDb.Summary()

Path: \\hpccluster\hpccluster-scratch\gutierrez\UnstrainedFlame_100ItMax_unitylew_noVariableKin_ConstantCP_highervel
Controller: BoSSS.Foundation.IO.DatabaseController
Sessions: System.Collections.Generic.List`1[BoSSS.Foundation.IO.ISessionInfo]
Grids: System.Collections.Generic.List`1[BoSSS.Foundation.IO.IGridInfo]
Projects: System.Collections.Generic.Dictionary`2[System.String,System.Collections.Generic.IEnumerable`1[BoSSS.Foundation.IO.ISessionInfo]]
AlternateDbPaths: System.ValueTuple`2[System.String,System.String][]


In [ ]:
BoSSSshell.WorkflowMgm.Init("ChamberedFlame4");
// BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.Sessions

In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[3];
// MiniBatchProcessor.Server.StartIfNotRunning(true);
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
// var myBatch = (SlurmClient)ExecutionQueues[1];
// var AddSbatchCmds = new List<string>();
// AddSbatchCmds.AddRange(new string[]{"#SBATCH -N 4","#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+2000});
// myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));
// myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
// myBatch.AdditionalBatchCommands

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int nCells) {
        double sepAdim = 20 ; // 20 is already ok!
        double xleft = -sepAdim;
        double xright = 0.0;
        double R = sepAdim / 10;

        var _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
        double[] _yNodes = GenericBlas.Linspace(-R, R, 3 + 1);
        string BC = "ScalarDirichlet_PressureOutlet";

        Console.WriteLine("Number of cells in the X direction: {0}", _xNodes.Length);
        Console.WriteLine("Number of cells in the Y direction: {0}", _yNodes.Length);
        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes, periodicY: true);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(2, BC);

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            if (Math.Abs(x - xleft) < 1e-8) { // Left boundary
                return 1; // Left
            } else if (Math.Abs(x - xright) < 1e-8) { // right boundary
                return 2; // right
            } else {
                return 3;
                //  throw new Exception("Problem while setting GridEdgeTagFunc");
            }
        }
        );
        

        myDb.SaveGrid(ref grd, true);

        return grd;
    }

}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal) {
        using(var stw = new System.IO.StringWriter()) {
            stw.WriteLine("static class BoundaryValues {");
            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");
            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }
    static public Formula Get_ConstantValue(double ConstVal){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal));
    }
}

## Create base control file
In this ControlFile basic configuration of the ChamberedDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();
    // C.AlternateDbPaths = new[] {
    //     (@"S:\work\scratch\jg11bano\"+dirname, "PCMIT30"),
    //     (@"/work/scratch/jg11bano/"+dirname,"")};
    // C.AlternateDbPaths = new[] {
    //     (winpath, ""),
    //     (dirname,"")};
//     C.AlternateDbPaths = new[]{
//         new ValueTuple<string,string>(@"/work/scratch/jg11bano/bosss_db", ""),
//         new ValueTuple<string,string>(@"Y:\bosss_db", "")
// };
    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //
    C.SetGrid(GridFactory.GenerateGrid(nCells));  //

    C.MatParamsMode = MaterialParamsMode.Sutherland;  //
    // Problem Definition
    //===================
    double TemperatureIn = 300;
    double massFuelIn = 0.24 * 0.1; //kg/m2s 
    double AtmPressure = 101325; // Pa


    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };
    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; 
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);

    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureIn * 1000); // Kg/m3. ok
    double uInFuel = massFuelIn / densityFuelIn; //
    Console.WriteLine("Velocity is "+uInFuel);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureIn;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = uInFuel; // m/s
    double rhoRef = pRef * mwAir / (8.314 * TRef * 1000); // Kg/m3. ok ;
    double DRef = 2.2133029473872009E-05 / rhoRef; 
    double LRef =  DRef / uRef; 
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.smoothingFactor = 80*0-1*1; 
    // C.NonLinearSolver.ConvergenceCriterion = 1e-8;
    // C.LinearSolver.ConvergenceCriterion = 1e-10;

    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();
    C.LinearSolver.verbose = false;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    C.PenaltyHeatConduction = 1.0;///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TRef;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds = C.rhoRef * C.uRef * C.LRef / C.muRef;
    C.Prandtl = 0.75;////////////////////0.75;
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    // // C.Lewis = new double[] { 0.97, 1.11, 1.39, 0.83, 1.0 };
    C.Lewis = new double[] {1.0, 1.0, 1.0,1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; // NOTE! activation temperature is also nondimensional
    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0));
    //==========================
    // Boundary conditions
    //==========================

    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(uInFuel / C.uRef));
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(0.0));
        


    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "ChamberedFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 50;
    
    // Boundary and initial conditions
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0));



    C_MixtureFraction.AdaptiveMeshRefinement = false;
   
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
 


    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:
bool HLLRCalculation = true;


In [ ]:

foreach(int nCells in NofCells){
int dg = 2;
    Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
    var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dg, nCells);
    string jobName       = C_MixtureFraction.SessionName;
    Console.WriteLine(jobName);
    var oneJob           = new Job(jobName, solver_MF);
    oneJob.NumberOfMPIProcs =  4;    
    //  oneJob.UseComputeNodesExclusive = true;
    oneJob.SetControlObject(C_MixtureFraction);
    oneJob.Activate(myBatch); 

}


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, ISessionInfo SessionToRestart) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "ChamberedFlame";
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = false; ///////////////////////

    // C_OneStep.Tags.Add("VelocityMultiplier" + mult);

    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C_OneStep.LinearSolver.NoOfMultigridLevels = 5;
    C_OneStep.ChemicalReactionActive = true;
    C_OneStep.AdaptiveMeshRefinement = false;
    
    C_OneStep.HeatCapacityMode = MaterialLaw_MultipleSpecies.CpCalculationMode.constant;

    C_OneStep.NoOfTimesteps = 1; // The steady solution will be calculated again and do AMR
    C_OneStep.NonLinearSolver.MaxSolverIterations = 50;


    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-1, 1.0 + 1e-1)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-1, 1.0 + 1e-1));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-1, 1.0 + 1e-1));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-1, 1.0 + 1e-1));
    C_OneStep.VariableBounds = Bounds;
    // Boundary conditions

    double dummy = 0;
    if(SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0));
    }

    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0]));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1]));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2]));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3]));

    C_OneStep.AddBoundaryValue("ScalarDirichlet_PressureOutlet", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0));
    C_OneStep.AddBoundaryValue("ScalarDirichlet_PressureOutlet", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0]));
    C_OneStep.AddBoundaryValue("ScalarDirichlet_PressureOutlet", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1]));
    C_OneStep.AddBoundaryValue("ScalarDirichlet_PressureOutlet", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2]));
    C_OneStep.AddBoundaryValue("ScalarDirichlet_PressureOutlet", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3]));
    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);NofCells

index,value
0,8
1,16
2,32
3,64
4,128
5,256
6,512


Calculate the full solution for the initial value

In [ ]:
int counter = 0;
foreach (int nCells in NofCells) {
    foreach (int dg in dgDegree) {
        // var sess =(myDb.Sessions.Where(s=>Convert.ToInt64(s.Tags.ToArray()[0]) == i)).FirstOrDefault(); 
        var sess = (myDb.Sessions.Where(s => s.Name == "FS_ChamberedFlameP" + 2 + "K" + nCells)).FirstOrDefault();
        var C = GiveMeTheFullCtrlFile(dg, nCells,  sess);
        string jobName = C.SessionName + "AllOn";
        Console.WriteLine(jobName);
        var oneJob = new Job(jobName, solver);
        oneJob.NumberOfMPIProcs = nCells > 32? 12:4 ;
        oneJob.SetControlObject(C);
        oneJob.Activate(myBatch);
        counter++;
    }
}


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Postprocessing

In [ ]:
// Now the flame sheet estimations should be deleted
BoSSSshell.WorkflowMgm.Sessions.Where(s => s.Name.StartsWith("FS_")).ForEach(x=>x.Delete(true));

In [ ]:
//WorkflowMgm.ResetSessionsCache();

In [ ]:
string basepath = @"C:\tmp\UnstrainedFlame_ConvStudy";
System.IO.Directory.CreateDirectory(basepath); // Create folder for storing data

In [ ]:
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure",
"MassFraction0","MassFraction1","MassFraction2","MassFraction3" }; 

In [ ]:
foreach (var varname in varnames) {
    foreach (int pDeg in dgDegree) {
       var pDegSessions = myDb.Sessions.Where(
            Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg 
        && (Convert.ToInt32(Si.KeysAndQueries["Grid:NoOfCells"]))/ 3 > 31
            ) // function which is true on all sessions we are interested in: 
         ).ToArray();
        Plot2Ddata pDegPlot =
         pDegSessions.ToEstimatedGridConvergenceData(varname,
           xAxis_Is_hOrDof: true, // false selects DOFs for x-axis 
           normType: NormType.L2_embedded );
            //Saving to a txt file
        pDegPlot.SaveTextFileToPublish(basepath+"\\"+  varname + "DG" + pDeg,false);
    }
}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.ITimestepInfoExtensions.<>c__DisplayClass24_0.<ToEstimatedGridConvergenceData>b__0(ITimestepInfo t) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\ITimestepInfoExtensions.cs:line 683
   at System.Linq.Lookup`2.Create(IEnumerable`1 source, Func`2 keySelector, IEqualityComparer`1 comparer)
   at System.Linq.GroupedEnumerable`2.GetEnumerator()
   at BoSSS.Application.BoSSSpad.ITimestepInfoExtensions.ToEstimatedGridConvergenceData(IEnumerable`1 timesteps, String fieldName, Boolean xAxis_Is_hOrDof, NormType normType) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\ITimestepInfoExtensions.cs:line 683
   at Submission#31.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentE

In [ ]:
myDb.Sessions

#0: ChamberedFlame4	Full_ChamberedFlameP4K512	09/26/2021 19:59:56	767b6a69...
#1: ChamberedFlame4	Full_ChamberedFlameP3K512	09/26/2021 19:38:17	a0f91d23...
#2: ChamberedFlame4	Full_ChamberedFlameP4K256	09/26/2021 19:24:08	0fa16dea...
#3: ChamberedFlame4	Full_ChamberedFlameP2K512	09/26/2021 19:26:11	83ab188c...
#4: ChamberedFlame4	Full_ChamberedFlameP3K256	09/26/2021 19:20:34	9261554c...
#5: ChamberedFlame4	Full_ChamberedFlameP1K512	09/26/2021 19:26:07	d50aecc9...
#6: ChamberedFlame4	Full_ChamberedFlameP2K256	09/26/2021 19:17:24	616a4f01...
#7: ChamberedFlame4	Full_ChamberedFlameP4K128	09/26/2021 19:13:51	8d6ec65b...
#8: ChamberedFlame4	Full_ChamberedFlameP1K256	09/26/2021 19:15:23	937faa73...
#9: ChamberedFlame4	Full_ChamberedFlameP3K128	09/26/2021 19:11:47	70123196...
#10: ChamberedFlame4	Full_ChamberedFlameP1K128	09/26/2021 19:06:12	5790b701...
#11: ChamberedFlame4	Full_ChamberedFlameP2K128	09/26/2021 19:07:24	5521442b...
#12: ChamberedFlame4	Full_ChamberedFlameP4K64	09/26/2021 19:03